In [25]:
import random

import polars as pl


In [26]:
# Create a DataFrame with a column called index and data fill with 15 records
df1 = pl.LazyFrame(
    {
        "index": range(15),
        "data": range(15),
    }
)

In [27]:
df1.collect()

index,data
i64,i64
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8


In [105]:
# Create a new DataFrame with a column called index fill with 4 records with random non conflicting values between 0-15
df2 = pl.LazyFrame(
    {
        "excluded_id": [2, 11, 13],
    }
)

In [106]:
df2.collect()

excluded_id
i64
2
11
13


In [107]:
batch_size = 3
# create list with 0-14 and shuffle it
indices = list(range(df1.collect().shape[0]-df2.collect().shape[0]))
random.shuffle(indices)
indices

[11, 8, 1, 3, 0, 5, 2, 10, 6, 9, 7, 4]

In [114]:
def sanitise_indices(indices, val_len, df2):
    ret_array = []
    # check if the index is in the second dataframe
    for i, idx in enumerate(indices):
        # get the row number of the index in df2

        is_sanitised = False
        ret_idx = idx
        while not is_sanitised:
            matches = df2.with_row_index(offset=0).filter(pl.col("excluded_id") == ret_idx).collect()
            if matches.shape[0] > 0:
                ret_idx = val_len + matches["index"][0]
            else:
                is_sanitised = True
                ret_array.append(ret_idx)
    return ret_array

In [115]:
indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
sanitise_indices(indices, df1.collect().shape[0]-df2.collect().shape[0], df2)

[0, 1, 12, 3, 4, 5, 6, 7, 8, 9, 10, 14]

In [121]:
# generator that yields the indices in batches
def batch_indices(batch_size, indices):
    for i in range(0, len(indices), batch_size):
        ret_indices = indices[i:i+batch_size]
        ret_indices = sanitise_indices(ret_indices, df1.collect().shape[0]-df2.collect().shape[0], df2)
        yield ret_indices

In [122]:
indices = list(range(df1.collect().shape[0]-df2.collect().shape[0]))
for batch in batch_indices(batch_size, indices):
    print(batch)

[0, 1, 12]
[3, 4, 5]
[6, 7, 8]
[9, 10, 14]
